In [60]:
!pip install krwordrank
!pip install kiwipiepy

In [123]:
import pandas as pd

df = pd.read_csv('./2022-04-18 news data.csv')
df

,Unnamed: 0,카테고리,언론사,기사제목,날짜,본문,기사링크
0,0,정치,SBS,[단독] '6억 집을 1억에'…이창용 한은 총재 후보자 다운계약 의혹,2022.04.18.,"\n\n\n\n\n이창용 한국은행 총재 후보자가 지난 2002년, 서울 중구 신당동...",https://news.naver.com//main/read.naver?mode=L...
1,1,정치,중앙일보,"노정희 선관위원장, '소쿠리 대선' 논란 44일만에 사퇴",2022.04.18.,\n\n\n\n\n\t\n\t 노정희 중앙선거관리위원회위원장이 18일 사의를 표명...,https://news.naver.com//main/read.naver?mode=L...
2,2,정치,YTN,단거리 발사에 김정은 참석...전술핵 탑재 노렸나,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]김정은 국무위원장이 단...,https://news.naver.com//main/read.naver?mode=L...
3,3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...
4,4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...
...,...,...,...,...,...,...,...
4641,4641,세계,국제신문,전 세계 물가 폭등 아우성…30년만에 ‘大인플레’ 시대 진입,2022.04.18.,\n\n\n\n\nOECD 2월 상승률 7.7% 급등- 우크라發 에너지가격 큰 폭 ...,https://news.naver.com//main/read.naver?mode=L...
4642,4642,세계,국민일보,물·전기 없이 13만명 ‘절망의 항전’… 마리우폴 ‘참극’,2022.04.18.,\n\n\n\n\n초토화 작전에 건물 90% 이상 파괴우크라군 최악 포위전 속 결사...,https://news.naver.com//main/read.naver?mode=L...
4643,4643,세계,중앙일보,러 “마리우폴 함락” 주장…우크라 “방어군 버티고 있다”,2022.04.18.,\n\n\n\n\n 우크라이나 동남부 돈바스 지역에서 친러 분리주의 무장대원들이 ...,https://news.naver.com//main/read.naver?mode=L...
4644,4644,세계,중앙일보,"침몰한 러의 자존심…미 “푸틴, 핵 쓸 가능성 커져”",2022.04.18.,\n\n\n\n\n\t\n\t 지난 14일 러시아 해군 흑해함대의 기함인 모스크바...,https://news.naver.com//main/read.naver?mode=L...


In [124]:
from krwordrank.word import summarize_with_keywords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
from kiwipiepy import Kiwi, Match
from kiwipiepy.utils import Stopwords

def clean_text(title):
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※%~∼ㆍ!【】』㈜©囹圄秋 ■◆◇▷▶◁◀ △▲▽▼<>‘|\(\)\[\]`\'…》→←↑↓↔〓♤♠♡♥♧♣⊙◈▣◐◑☆★\”\“\’·※~ ! @ # $ % ^ & * \ " ]', ' ', title)
    text_rmv = ' '.join(text_rmv.split())
    return text_rmv # 문자열

def noun_extractor(sentence):
    nouns =[]
    kiwi = Kiwi()
    stopwords = Stopwords()
    result_iter = kiwi.tokenize(sentence, normalize_coda=True,stopwords=stopwords)
    i = 0
    for i in range(len(result_iter)):
        if result_iter[i][1] == 'NNP':
            nouns.append(result_iter[i][0])
    return nouns # 리스트로 반환

def listToString(str_list): #리스트를 문자열로 반환 (특수문자 제거 후 추출된 명사 입력)
    result = ""
    for s in str_list:
        result += s + " "
    string = result.strip()
    return string

def kwordrank(string): #해당 함수에서 불용어 제거 
    stop_words_list=[]
    stop_words_file = open("korean_stop_words.txt",encoding='utf-8')
    for line in stop_words_file.readlines():
        stop_words_list.append(line.rstrip())
        stop_words_file.close()
    stopwords=stop_words_list
    texts = [string]
    keywords = summarize_with_keywords(texts, min_count=1, max_length=10,beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
    passwords = {word:score for word, score in sorted(
        keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}
    return passwords

In [125]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import numpy as np

def show_relevant_article(category, df):     # 사용자에게 카테고리와 데이터를 인자로 입력받음
    df = df[df['카테고리']==category].reset_index()

    # 기사 본문 클러스터링
    text = list(df.기사제목) # 본문보다 효과가 좋은 것 같음. 본문으로 돌리려면 df.본문이라고 수정하면 됨

    lst = []
    for i in range(len(text)):
        text[i] = clean_text(text[i])
        lst.append(text[i])
    
    df['cleaned_제목'] = lst

    content = df['cleaned_제목'].tolist()
    n_clusters = 30 # 군집화할 갯수
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(content)
    X = normalize(X)
    kmeans = KMeans(n_clusters=n_clusters).fit(X)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    df['labels'] = labels
    data = df.labels # 숫자 라벨 생성
    labels_count = data.value_counts()
    
    # kwrodrank로 라벨 키워드 뽑기 
    top_word_30 = []
    for i in range(len(labels_count)):
        df_i = df[df.labels==i]

        df_i_content = list(df_i.cleaned_제목)
        df_content_del = listToString(df_i_content)

        k = kwordrank(df_content_del)
        k_1 = list(k.keys())[0]
        top_word_30.append(k_1)

    # 숫자 라벨과 kwordrank로 추출한 키워드를 replace
    for i in range(len(labels_count)):
        for j in range(len(data)):
            if i == data[j]:
                data[j] = top_word_30[i]

    df = df.drop('cleaned_제목', axis = 1)
    df = df.drop('index', axis = 1)
    df = df.drop('Unnamed: 0', axis = 1)

    return df

In [130]:
show_relevant_article('생활', df)

scan vocabs ... 
num vocabs = 2358
done
scan vocabs ... 
num vocabs = 35
done
scan vocabs ... 
num vocabs = 124
done
scan vocabs ... 
num vocabs = 62
done
scan vocabs ... 
num vocabs = 176
done
scan vocabs ... 
num vocabs = 104
done
scan vocabs ... 
num vocabs = 127
done
scan vocabs ... 
num vocabs = 91
done
scan vocabs ... 
num vocabs = 103
done
scan vocabs ... 
num vocabs = 149
done
scan vocabs ... 
num vocabs = 98
done
scan vocabs ... 
num vocabs = 99
done
scan vocabs ... 
num vocabs = 146
done
scan vocabs ... 
num vocabs = 74
done
scan vocabs ... 
num vocabs = 38
done
scan vocabs ... 
num vocabs = 73
done
scan vocabs ... 
num vocabs = 229
done
scan vocabs ... 
num vocabs = 103
done
scan vocabs ... 
num vocabs = 141
done
scan vocabs ... 
num vocabs = 93
done
scan vocabs ... 
num vocabs = 144
done
scan vocabs ... 
num vocabs = 127
done
scan vocabs ... 
num vocabs = 172
done
scan vocabs ... 
num vocabs = 140
done
scan vocabs ... 
num vocabs = 208
done
scan vocabs ... 
num vocabs = 171

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,카테고리,언론사,기사제목,날짜,본문,기사링크,labels
0,생활,헬스조선,"희귀질환 '파브리병', 어떤 질병인가요?",2022.04.18.,\n\n\n\n\n김영도 가톨릭대학교 인천성모병원 뇌병원 교수​/인천성모병원 제공낯...,https://news.naver.com//main/read.naver?mode=L...,저도
1,생활,연합뉴스TV,[생활날씨] 내일 큰 일교차…곳곳 짙은 안개·먼지,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t이제 낮에는 반소매 차림도 제...,https://news.naver.com//main/read.naver?mode=L...,날씨
2,생활,국민일보,"코로나 앓았던 아이, 이전보다 땀 많고 자꾸 누우려고 하면 ‘무기력증’ 의심",2022.04.18.,\n\n\n\n\n마른 기침엔 배·도라지즙 권장식욕 부진 심할 땐 침·뜸이 좋아함...,https://news.naver.com//main/read.naver?mode=L...,코로나
3,생활,서울신문,선 넘었다…‘태극기’에 바이러스 CG 합성한 방송,2022.04.18.,\n\n\n\n\n대만 방송국 TVBS 뉴스가 한국의 코로나19 관련 보도를 하면서...,https://news.naver.com//main/read.naver?mode=L...,저도
4,생활,매일경제,"""그 돈이면 '벤츠' 사지""…'반쪽 전기차' 깔봤는데, '반전매력' GV70 EV ...",2022.04.18.,\n\n\n\n\nGV70 완성도 높인 전동화모델모래·진흙·눈에 강한 e-터레인렉서...,https://news.naver.com//main/read.naver?mode=L...,전기차
...,...,...,...,...,...,...,...
219,생활,중앙일보,[건강한 가족] 피로·무력감? 홍삼 꾸준히 섭취하면 개선 효과,2022.04.18.,\n\n\n\n\n ━ 홍삼의 효능 눈길 코로나19...,https://news.naver.com//main/read.naver?mode=L...,가족
220,생활,중앙일보,뇌도 공격하는 만성 외로움...날 도와줄 친구 없다? 의외 방법 [건강한 가족],2022.04.18.,\n\n\n\n\n가족·지인과 전화·SNS 연락 유지취미·등산 모임에 활발하게 참가...,https://news.naver.com//main/read.naver?mode=L...,가족
221,생활,중앙일보,"""몸뿐 아니라 뇌·마음도 아프면 빨리 치료받는다는 인식 중요"" [건강한 가족]",2022.04.18.,\n\n\n\n\n어떤 병이든 초기에 치료해야회복 빠르고 손실 줄일 수 있어정신 질...,https://news.naver.com//main/read.naver?mode=L...,가족
222,생활,중앙일보,손톱깎이로 뜯으면 더 크게 생기네...여태 몰랐던 '굳은살 법칙' [건강한 가족],2022.04.18.,"\n\n\n\n\n지나친 각질 제거, 2차 감염 우려굳은살엔 샤워 뒤 보습제 듬뿍사...",https://news.naver.com//main/read.naver?mode=L...,가족
